In [ ]:
!pip install yfinance pandas numpy plotly ta scikit-learn

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ta
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
class AdvancedTradingSystem:
    def __init__(self):
        self.params = {
            # Moving Averages
            'SMA_SHORT': 20,
            'SMA_LONG': 50,
            'EMA_SHORT': 12,
            'EMA_LONG': 26,

            # RSI
            'RSI_PERIOD': 14,
            'RSI_OVERBOUGHT': 70,
            'RSI_OVERSOLD': 30,

            # MACD
            'MACD_FAST': 12,
            'MACD_SLOW': 26,
            'MACD_SIGNAL': 9,

            # Bollinger Bands
            'BB_PERIOD': 20,
            'BB_STD': 2,

            # Volume
            'VOLUME_MA': 20,

            # Risk Management
            'STOP_LOSS': 0.02,
            'TAKE_PROFIT': 0.03
        }

    def fetch_data(self, symbol, period='6mo', interval='1d'):
        """Fetch stock data and calculate indicators"""
        try:
            stock = yf.Ticker(symbol)
            df = stock.history(period=period, interval=interval)
            if not df.empty:
                return self.add_indicators(df)
            return None
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None

    def add_indicators(self, df):
        """Add technical indicators to dataframe"""
        # Moving Averages
        df['SMA_short'] = ta.trend.sma_indicator(df['Close'],
                                                self.params['SMA_SHORT'])
        df['SMA_long'] = ta.trend.sma_indicator(df['Close'],
                                               self.params['SMA_LONG'])
        df['EMA_short'] = ta.trend.ema_indicator(df['Close'],
                                                self.params['EMA_SHORT'])
        df['EMA_long'] = ta.trend.ema_indicator(df['Close'],
                                               self.params['EMA_LONG'])

        # RSI
        df['RSI'] = ta.momentum.rsi(df['Close'], self.params['RSI_PERIOD'])

        # MACD
        macd = ta.trend.MACD(df['Close'],
                            self.params['MACD_FAST'],
                            self.params['MACD_SLOW'],
                            self.params['MACD_SIGNAL'])
        df['MACD'] = macd.macd()
        df['MACD_signal'] = macd.macd_signal()
        df['MACD_hist'] = macd.macd_diff()

        # Bollinger Bands
        bb = ta.volatility.BollingerBands(df['Close'],
                                         self.params['BB_PERIOD'],
                                         self.params['BB_STD'])
        df['BB_upper'] = bb.bollinger_hband()
        df['BB_lower'] = bb.bollinger_lband()
        df['BB_middle'] = bb.bollinger_mavg()

        # Volume
        df['Volume_MA'] = df['Volume'].rolling(window=self.params['VOLUME_MA']).mean()
        df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']

        return df.dropna()

    def generate_signals(self, df):
      signals = pd.DataFrame(index=df.index)
      signals['Signal'] = 0

      try:
          # Buy Conditions (More relaxed to generate more buy signals)
          buy_conditions = (
              # Primary Trend Condition (need only one)
              (
                  (df['Close'] > df['SMA_short']) |  # Uptrend
                  (df['MACD'] > df['MACD_signal'])   # MACD bullish
              ) &

              # Secondary Conditions (need at least one)
              (
                  (df['RSI'] < 40) |                # Relaxed RSI oversold
                  (df['Close'] < df['BB_lower']) |  # Price below lower BB
                  (df['Volume_Ratio'] > 1.1)        # Relaxed volume requirement
              ) &

              # Additional confirmation (optional)
              (
                  (df['Close'] > df['Close'].shift(1)) |  # Price increasing
                  (df['Volume'] > df['Volume'].shift(1))  # Volume increasing
              )
          )

          # Sell Conditions (Kept strict)
          sell_conditions = (
              (df['RSI'] > 75) &                     # Overbought
              (df['Close'] < df['SMA_short']) &      # Below short MA
              (df['MACD'] < df['MACD_signal'])       # MACD bearish
          )

          signals.loc[buy_conditions, 'Signal'] = 1
          signals.loc[sell_conditions, 'Signal'] = -1

          return signals

      except Exception as e:
          print(f"Error generating signals: {e}")
          return signals

    def plot_analysis(self, df, signals, symbol):
        """Create high-resolution analysis plot"""
        fig = make_subplots(
            rows=4, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.03,
            row_heights=[0.5, 0.2, 0.15, 0.15],
            subplot_titles=('Price & Signals', 'Volume', 'RSI', 'MACD')
        )

        # Price and indicators
        fig.add_trace(
            go.Candlestick(
                x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name='Price'
            ), row=1, col=1
        )

        # Add moving averages
        fig.add_trace(
            go.Scatter(x=df.index, y=df['SMA_short'],
                      name='SMA20', line=dict(color='orange')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['SMA_long'],
                      name='SMA50', line=dict(color='blue')),
            row=1, col=1
        )

        # Add Bollinger Bands
        fig.add_trace(
            go.Scatter(x=df.index, y=df['BB_upper'],
                      name='BB Upper', line=dict(color='gray', dash='dash')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['BB_lower'],
                      name='BB Lower', line=dict(color='gray', dash='dash')),
            row=1, col=1
        )

        # Add buy signals
        buy_signals = signals[signals['Signal'] == 1]
        fig.add_trace(
            go.Scatter(
                x=buy_signals.index,
                y=df.loc[buy_signals.index]['Low'] * 0.99,
                mode='markers',
                name='Buy Signal',
                marker=dict(symbol='triangle-up', size=15, color='green')
            ), row=1, col=1
        )

        # Add sell signals
        sell_signals = signals[signals['Signal'] == -1]
        fig.add_trace(
            go.Scatter(
                x=sell_signals.index,
                y=df.loc[sell_signals.index]['High'] * 1.01,
                mode='markers',
                name='Sell Signal',
                marker=dict(symbol='triangle-down', size=15, color='red')
            ), row=1, col=1
        )

        # Volume
        fig.add_trace(
            go.Bar(x=df.index, y=df['Volume'], name='Volume'),
            row=2, col=1
        )

        # RSI
        fig.add_trace(
            go.Scatter(x=df.index, y=df['RSI'], name='RSI'),
            row=3, col=1
        )
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

        # MACD
        fig.add_trace(
            go.Bar(x=df.index, y=df['MACD_hist'], name='MACD Histogram'),
            row=4, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['MACD'], name='MACD'),
            row=4, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['MACD_signal'], name='Signal Line'),
            row=4, col=1
        )

        # Update layout for high resolution
        fig.update_layout(
            title=f'{symbol} Technical Analysis',
            width=1920,
            height=1080,
            template='plotly_dark',
            showlegend=True,
            xaxis_rangeslider_visible=False
        )

        fig.show()

    def print_analysis(self, df, symbol):
        """Print current market analysis"""
        latest = df.iloc[-1]

        print(f"\n=== Analysis for {symbol} ===")
        print(f"Date: {latest.name.strftime('%Y-%m-%d')}")
        print(f"Current Price: ₹{latest['Close']:.2f}")

        print("\nTechnical Indicators:")
        print(f"RSI: {latest['RSI']:.2f}")
        print(f"MACD: {latest['MACD']:.2f}")
        print(f"MACD Signal: {latest['MACD_signal']:.2f}")
        print(f"Volume Ratio: {latest['Volume_Ratio']:.2f}x")

        # Generate signal
        signal = "HOLD"
        if (latest['RSI'] < self.params['RSI_OVERSOLD'] and
            latest['Close'] > latest['SMA_short'] and
            latest['MACD'] > latest['MACD_signal']):
            signal = "BUY"
        elif (latest['RSI'] > self.params['RSI_OVERBOUGHT'] or
              latest['Close'] < latest['SMA_short'] or
              latest['MACD'] < latest['MACD_signal']):
            signal = "SELL"

        print(f"\nCurrent Signal: {signal}")

        print("\nKey Levels:")
        print(f"Support (BB Lower): ₹{latest['BB_lower']:.2f}")
        print(f"Resistance (BB Upper): ₹{latest['BB_upper']:.2f}")
        print(f"Stop Loss: ₹{latest['Close'] * (1 - self.params['STOP_LOSS']):.2f}")
        print(f"Take Profit: ₹{latest['Close'] * (1 + self.params['TAKE_PROFIT']):.2f}")

In [ ]:
class AdvancedTradingSystem:
    def __init__(self):
        self.params = {
    # Moving Averages
    'SMA_SHORT': 20,
    'SMA_LONG': 50,
    'EMA_SHORT': 12,
    'EMA_LONG': 26,

    # RSI Parameters (Adjusted)
    'RSI_PERIOD': 14,
    'RSI_OVERSOLD': 40,    # Increased from 35
    'RSI_OVERBOUGHT': 75,  # Kept same

    # MACD Parameters
    'MACD_FAST': 12,
    'MACD_SLOW': 26,
    'MACD_SIGNAL': 9,

    # Bollinger Bands
    'BB_PERIOD': 20,
    'BB_STD': 2,

    # Volume (Relaxed)
    'VOLUME_MA': 15,
    'MIN_VOLUME_RATIO': 1.1,  # Decreased from 1.2

    # Risk Management
    'STOP_LOSS': 0.02,
    'TAKE_PROFIT': 0.04
}

    def fetch_data(self, symbol, period='6mo', interval='1d'):
        """Fetch stock data and calculate indicators"""
        try:
            stock = yf.Ticker(symbol)
            df = stock.history(period=period, interval=interval)
            if not df.empty:
                return self.add_indicators(df)
            return None
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None

    def add_indicators(self, df):
        """Add technical indicators to dataframe"""
        # Moving Averages
        df['SMA_short'] = ta.trend.sma_indicator(df['Close'],
                                                self.params['SMA_SHORT'])
        df['SMA_long'] = ta.trend.sma_indicator(df['Close'],
                                               self.params['SMA_LONG'])
        df['EMA_short'] = ta.trend.ema_indicator(df['Close'],
                                                self.params['EMA_SHORT'])
        df['EMA_long'] = ta.trend.ema_indicator(df['Close'],
                                               self.params['EMA_LONG'])

        # RSI
        df['RSI'] = ta.momentum.rsi(df['Close'], self.params['RSI_PERIOD'])

        # MACD
        macd = ta.trend.MACD(df['Close'],
                            self.params['MACD_FAST'],
                            self.params['MACD_SLOW'],
                            self.params['MACD_SIGNAL'])
        df['MACD'] = macd.macd()
        df['MACD_signal'] = macd.macd_signal()
        df['MACD_hist'] = macd.macd_diff()

        # Bollinger Bands
        bb = ta.volatility.BollingerBands(df['Close'],
                                         self.params['BB_PERIOD'],
                                         self.params['BB_STD'])
        df['BB_upper'] = bb.bollinger_hband()
        df['BB_lower'] = bb.bollinger_lband()
        df['BB_middle'] = bb.bollinger_mavg()

        # Volume
        df['Volume_MA'] = df['Volume'].rolling(window=self.params['VOLUME_MA']).mean()
        df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']

        return df.dropna()

    def generate_signals(self, df):
        """Generate trading signals"""
        signals = pd.DataFrame(index=df.index)
        signals['Signal'] = 0  # 0: Hold, 1: Buy, -1: Sell

        # Buy conditions
        buy_conditions = (
            (df['RSI'] < self.params['RSI_OVERSOLD']) &  # Oversold
            (df['Close'] > df['SMA_short']) &            # Uptrend
            (df['MACD'] > df['MACD_signal']) &          # MACD crossover
            (df['Close'] < df['BB_lower']) &            # Below lower BB
            (df['Volume_Ratio'] > 1.5)                  # High volume
        )

        # Sell conditions
        sell_conditions = (
            (df['RSI'] > self.params['RSI_OVERBOUGHT']) |  # Overbought
            (df['Close'] < df['SMA_short']) |              # Downtrend
            (df['MACD'] < df['MACD_signal']) |            # MACD crossunder
            (df['Close'] > df['BB_upper'])                 # Above upper BB
        )

        signals.loc[buy_conditions, 'Signal'] = 1
        signals.loc[sell_conditions, 'Signal'] = -1

        return signals

    def plot_analysis(self, df, signals, symbol):
        """Create high-resolution analysis plot"""
        fig = make_subplots(
            rows=4, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.03,
            row_heights=[0.5, 0.2, 0.15, 0.15],
            subplot_titles=('Price & Signals', 'Volume', 'RSI', 'MACD')
        )

        # Price and indicators
        fig.add_trace(
            go.Candlestick(
                x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name='Price'
            ), row=1, col=1
        )

        # Add moving averages
        fig.add_trace(
            go.Scatter(x=df.index, y=df['SMA_short'],
                      name='SMA20', line=dict(color='orange')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['SMA_long'],
                      name='SMA50', line=dict(color='blue')),
            row=1, col=1
        )

        # Add Bollinger Bands
        fig.add_trace(
            go.Scatter(x=df.index, y=df['BB_upper'],
                      name='BB Upper', line=dict(color='gray', dash='dash')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['BB_lower'],
                      name='BB Lower', line=dict(color='gray', dash='dash')),
            row=1, col=1
        )

        # Add buy signals
        buy_signals = signals[signals['Signal'] == 1]
        fig.add_trace(
            go.Scatter(
                x=buy_signals.index,
                y=df.loc[buy_signals.index]['Low'] * 0.99,
                mode='markers',
                name='Buy Signal',
                marker=dict(symbol='triangle-up', size=15, color='green')
            ), row=1, col=1
        )

        # Add sell signals
        sell_signals = signals[signals['Signal'] == -1]
        fig.add_trace(
            go.Scatter(
                x=sell_signals.index,
                y=df.loc[sell_signals.index]['High'] * 1.01,
                mode='markers',
                name='Sell Signal',
                marker=dict(symbol='triangle-down', size=15, color='red')
            ), row=1, col=1
        )

        # Volume
        fig.add_trace(
            go.Bar(x=df.index, y=df['Volume'], name='Volume'),
            row=2, col=1
        )

        # RSI
        fig.add_trace(
            go.Scatter(x=df.index, y=df['RSI'], name='RSI'),
            row=3, col=1
        )
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

        # MACD
        fig.add_trace(
            go.Bar(x=df.index, y=df['MACD_hist'], name='MACD Histogram'),
            row=4, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['MACD'], name='MACD'),
            row=4, col=1
        )
        fig.add_trace(
            go.Scatter(x=df.index, y=df['MACD_signal'], name='Signal Line'),
            row=4, col=1
        )

        # Update layout for high resolution
        fig.update_layout(
            title=f'{symbol} Technical Analysis',
            width=1920,
            height=1080,
            template='plotly_dark',
            showlegend=True,
            xaxis_rangeslider_visible=False
        )

        fig.show()

    def print_analysis(self, df, symbol):
        """Print current market analysis"""
        latest = df.iloc[-1]

        print(f"\n=== Analysis for {symbol} ===")
        print(f"Date: {latest.name.strftime('%Y-%m-%d')}")
        print(f"Current Price: ₹{latest['Close']:.2f}")

        print("\nTechnical Indicators:")
        print(f"RSI: {latest['RSI']:.2f}")
        print(f"MACD: {latest['MACD']:.2f}")
        print(f"MACD Signal: {latest['MACD_signal']:.2f}")
        print(f"Volume Ratio: {latest['Volume_Ratio']:.2f}x")

        # Generate signal
        signal = "HOLD"
        if (latest['RSI'] < self.params['RSI_OVERSOLD'] and
            latest['Close'] > latest['SMA_short'] and
            latest['MACD'] > latest['MACD_signal']):
            signal = "BUY"
        elif (latest['RSI'] > self.params['RSI_OVERBOUGHT'] or
              latest['Close'] < latest['SMA_short'] or
              latest['MACD'] < latest['MACD_signal']):
            signal = "SELL"

        print(f"\nCurrent Signal: {signal}")

        print("\nKey Levels:")
        print(f"Support (BB Lower): ₹{latest['BB_lower']:.2f}")
        print(f"Resistance (BB Upper): ₹{latest['BB_upper']:.2f}")
        print(f"Stop Loss: ₹{latest['Close'] * (1 - self.params['STOP_LOSS']):.2f}")
        print(f"Take Profit: ₹{latest['Close'] * (1 + self.params['TAKE_PROFIT']):.2f}")

In [ ]:
trading_system = AdvancedTradingSystem()

# Define Nifty 50 symbols (add more as needed)
NIFTY50_SYMBOLS = [
    "RELIANCE.NS",
    "TCS.NS",
    "HDFCBANK.NS",
    "INFY.NS",
    "ICICIBANK.NS"
]

# Analyze each stock
for symbol in NIFTY50_SYMBOLS:
    print(f"\nAnalyzing {symbol}...")

    # Fetch and process data
    df = trading_system.fetch_data(symbol)

    if df is not None:
        # Generate signals
        signals = trading_system.generate_signals(df)

        # Print analysis
        trading_system.print_analysis(df, symbol)

        # Plot analysis
        trading_system.plot_analysis(df, signals, symbol)


Analyzing RELIANCE.NS...

=== Analysis for RELIANCE.NS ===
Date: 2025-03-06
Current Price: ₹1209.60

Technical Indicators:
RSI: 46.77
MACD: 17.40
MACD Signal: 14.46
Volume Ratio: 1.35x

Current Signal: SELL

Key Levels:
Support (BB Lower): ₹1161.33
Resistance (BB Upper): ₹1275.64
Stop Loss: ₹1185.41
Take Profit: ₹1257.98



Analyzing TCS.NS...

=== Analysis for TCS.NS ===
Date: 2025-03-06
Current Price: ₹3601.60

Technical Indicators:
RSI: 31.58
MACD: 142.06
MACD Signal: 125.61
Volume Ratio: 0.88x

Current Signal: SELL

Key Levels:
Support (BB Lower): ₹3399.11
Resistance (BB Upper): ₹4161.86
Stop Loss: ₹3529.57
Take Profit: ₹3745.66



Analyzing HDFCBANK.NS...

=== Analysis for HDFCBANK.NS ===
Date: 2025-03-06
Current Price: ₹1691.20

Technical Indicators:
RSI: 47.23
MACD: 1.42
MACD Signal: 0.65
Volume Ratio: 1.43x

Current Signal: SELL

Key Levels:
Support (BB Lower): ₹1670.24
Resistance (BB Upper): ₹1742.32
Stop Loss: ₹1657.38
Take Profit: ₹1758.85



Analyzing INFY.NS...

=== Analysis for INFY.NS ===
Date: 2025-03-06
Current Price: ₹1715.85

Technical Indicators:
RSI: 34.87
MACD: 46.10
MACD Signal: 35.58
Volume Ratio: 1.71x

Current Signal: SELL

Key Levels:
Support (BB Lower): ₹1661.50
Resistance (BB Upper): ₹1947.63
Stop Loss: ₹1681.53
Take Profit: ₹1784.48



Analyzing ICICIBANK.NS...

=== Analysis for ICICIBANK.NS ===
Date: 2025-03-06
Current Price: ₹1218.05

Technical Indicators:
RSI: 41.85
MACD: 11.18
MACD Signal: 8.58
Volume Ratio: 1.16x

Current Signal: SELL

Key Levels:
Support (BB Lower): ₹1196.41
Resistance (BB Upper): ₹1279.95
Stop Loss: ₹1193.69
Take Profit: ₹1266.77


In [ ]:
symbol = "RELIANCE.NS"
df = trading_system.fetch_data(symbol)

if df is not None:
    signals = trading_system.generate_signals(df)
    trading_system.print_analysis(df, symbol)
    trading_system.plot_analysis(df, signals, symbol)


=== Analysis for RELIANCE.NS ===
Date: 2025-03-06
Current Price: ₹1209.60

Technical Indicators:
RSI: 46.77
MACD: 17.40
MACD Signal: 14.46
Volume Ratio: 1.34x

Current Signal: SELL

Key Levels:
Support (BB Lower): ₹1161.33
Resistance (BB Upper): ₹1275.64
Stop Loss: ₹1185.41
Take Profit: ₹1245.89
